In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
spark = SparkSession.builder.appName('2021EDA').getOrCreate()

#change configuration settings on Spark 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '4g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','4g')])

#print spark configuration settings
spark.sparkContext.getConf().getAll()

In [ ]:
df_2021 = spark.read.csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/rideshare/2021", inferSchema=True, header=True)
# figure out how to read in shp file msca-bdp-student-gcs/bdp-rideshare-project/neighborhoods/shp files
df_weather = spark.read.csv("gs://msca-bdp-student-gcs/bdp-rideshare-project/weather/chicago 2020-01-01 to 2022-08-31.csv", inferSchema=True, header=True)
df_2021.printSchema()
df_weather.printSchema()

In [ ]:
#display number of records by partition
def displaypartitions(df):
    #number of records by partition
    num = df.rdd.getNumPartitions()
    print("Partitions:", num)
    df.withColumn("partitionId", F.spark_partition_id())\
        .groupBy("partitionId")\
        .count()\
        .orderBy(F.asc("count"))\
        .show(num)

df_2021.rdd.getNumPartitions()

In [ ]:
displaypartitions(df_2021)

In [ ]:
df_2021 = df_2021.repartition(10)

In [ ]:
displaypartitions(df_2021)

In [ ]:
df_2021.describe().show()

In [ ]:
#Find the number of missing values for each column
from pyspark.sql.functions import isnan, when, count, col
df_2021.select([count(when(df_2021[c].isNull(), c)).alias(c) for c in df_2021.columns]).show()

In [ ]:
# number of observations with all the data in each column
df_2021.dropna().count()

In [ ]:
# Working with just data that contains full information and check for dupes
df_2021 = df_2021.dropna()
df_2021 = df_2021.dropDuplicates()